# Create mapping to match tags from Zotero with tags from spreadsheet

## Import relevant modules

In [1]:
import requests
import pandas as pd
import re

## Define methods

In [2]:
def get_total_results(url):
    response = requests.get(url)
    total_results = response.headers["Total-Results"]
    print("Number of total results (according to the header): " + total_results)
    return int(total_results)

def get_tags(url):
    total_results = get_total_results(url)
    print("Retrieving all tags...")
    all_responses = []
    start = 0
    while start <= total_results:
        print(start, end=" ")
        params = {'start': start}
        response = requests.get(url, params=params)
        all_responses.extend(response.json())
        start += 100
    print("Number of tags retrieved: " + str(len(all_responses)))
    return all_responses

## Get tags from Zotero

### Get tags with /tags request

In [3]:
url = "https://api.zotero.org/groups/4822236/tags?format=json&include=data&sort=title&direction=asc&limit=100"

tags = get_tags(url)

Number of total results (according to the header): 1824
Retrieving all tags...
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 Number of tags retrieved: 1824


In [4]:
tags_list = [tag["tag"] for tag in tags]

print(len(tags_list))

print(tags_list[:5])

1824
['_TODO', "'2001: A Space Odyssey' (film)", "'American Historical Review'", "'History Matters'", "'Journal of American History'"]


### Get tags with /items/tags request

In [5]:
url_items_tags = "https://api.zotero.org/groups/4822236/items/tags?format=json&include=data&sort=title&direction=asc&limit=100"

items_tags = get_tags(url_items_tags)

Number of total results (according to the header): 1823
Retrieving all tags...
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 Number of tags retrieved: 1823


In [6]:
items_tags_list = [tag["tag"] for tag in items_tags]

print(len(items_tags_list))

print(items_tags_list[:5])

1823
['_TODO', "'2001: A Space Odyssey' (film)", "'American Historical Review'", "'History Matters'", "'Journal of American History'"]


### Find tag missing in /items/tags response

In [7]:
for tag in tags_list:
    if tag not in items_tags_list:
        print(tag)

Foreign Language Study / German


More info on this tag can be retrieved by sending the request https://api.zotero.org/groups/4822236/tags?tag=Foreign%20Language%20Study%20%2F%20German. The tag is used for the book "Digital Literary Studies: Corpus Approaches to Poetry, Prose, and Drama".

Let's also check the other way around.

In [8]:
for tag in items_tags_list:
    if tag not in tags_list:
        print(tag)

Apparently, there are no extra tags in `items_tags_list`.

## Get tags from spreadsheet

We will read directly from a CSV export of the Google Sheets spreadsheet. A copy of the CSV is provided in the same folder as this notebook.

Direct integration with Google Sheets does not seems to be necessary at the moment, but could be included in a future revision of this script.

In [9]:
df = pd.read_csv('zotero_keywords.csv')

df

,Cleaned_Version (new),Tag (existent),Other variants (= other existent tags),German variants (already existent),Count,German translation,Broader concept,Susanne Cat 1,Susanne Cat 2,Susanne Cat 3,...,new_2,cat_2,new_3,cat_3,new_4,cat_4,new_5,cat_5,new_6,cat_6
0,(semi-)automated generated,(teil-)automatisch generiert,NaN,(teil-)automatisch generiert,4.0,(teil-)automatisch generiert,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,#nosource,NaN,NaN,4.0,NaN,NaN,object-type,resource/tool,topic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,medieval england (1066-1485),1066-1485,NaN,NaN,1.0,1066-1485,NaN,time period,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1922,NaN,NaN,1.0,1922,NaN,time period,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2001: a space odyssey (film),NaN,NaN,1.0,NaN,NaN,entity (product),topic,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,NaN,x-check,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,extensible markup language (xml),xml,xml schema,NaN,16.0,XML,NaN,format/standard,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,year in dh 2008,yearindh2008,NaN,NaN,1.0,DH-Jahr 2008,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,NaN,youth,NaN,NaN,1.0,Jugend,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will now list all columns with their respective index.

In [10]:
pd.Series(df.columns)

0                      Cleaned_Version (new)
1                             Tag (existent)
2     Other variants (= other existent tags)
3         German variants (already existent)
4                                      Count
5                         German translation
6                            Broader concept
7                              Susanne Cat 1
8                              Susanne Cat 2
9                              Susanne Cat 3
10                                Nina Cat 1
11                                Nina Cat 2
12                              Martin Cat 1
13                              Martin Cat 2
14                              Martin Cat 3
15                            not ger or eng
16                                    delete
17                            delete_textual
18                                     split
19                                     new_1
20                                     cat_1
21                                     new_2
22        

Only part of these columns are necessary for our purpose. We can therefore drop some of them to make the processing of the dataframe easier.

In [11]:
df.drop(df.columns[6:15], axis=1, inplace=True)

We will now get a list of all the tags that were in the Zotero library according to the spreadsheet.

* `Cleaned_Version (new)`: each cells contains only *one* single tag. The separators present here are part of the *same* tag.
* `Tag (existent)`: each cell contains one single tag here too.
* `Other variants (= other existent tags)`: separators `,` and `;` are used. The problem is that sometimes the separator `,` is part of a single tag. I should have now uniformed all instances, so that `;` is the only separator that is being used *between two tags* (other separators might appear inside a single tag).
* `German variants (already existent)`: separator `,` is part of the tag (as in `Architektur,`). One **question** regarding `estadísticas lexicales, lexical statistics`. Are these two separate tags? The second one does not appear in the Zotero library. In any case, for this column we can just specify `;` as separator.

In [12]:
tags_spreadsheet = []

# Tag (existent)
tags_spreadsheet.extend(df.iloc[:,1].dropna().values.tolist())
# Other variants (= other existent tags)
tags_spreadsheet.extend([item for sublist in df.iloc[:,2].dropna().apply(lambda x: x.split('; ')).values.tolist() for item in sublist])
# German variants (already existent)
tags_spreadsheet.extend([item for sublist in df.iloc[:,3].dropna().apply(lambda x: x.split('; ')).values.tolist() for item in sublist])

print(len(tags_spreadsheet))
print(tags_spreadsheet)

1622
['(teil-)automatisch generiert', '#nosource', '1066-1485', '1922', '2001: a space odyssey (film)', '2011', '3d', '550 geowissenschaften', 'abe', 'academia', 'academic', 'academics', 'acceso', 'active learning', 'activism', 'adafruit', 'adam', 'address book', 'administration', 'administration technological innovations', 'advocating', 'africa', 'african american', 'age', 'agenda', 'agora', 'agriculture', 'ailion', 'aims news', 'àlex', 'algorithm design and analysis', 'alphabetisch katalogisieren', 'altered books', 'american', 'american historical association', 'american historical review', 'american history', 'american social history project', 'amerikaanse letterkunde', 'analyse des données', 'analysis', 'analytics', 'analyze qualitatively', 'analyze quantitatively', 'ancient', 'android', 'animation', 'anno 1900-1999', 'annotation', 'annotation system design', 'anthony', 'anthropology', 'anyobject', 'app', 'applications and critiques', 'apportionment', 'arch', 'archaeologicalsites',

Therefore, the tags retrived from Zotero are **1824**, while those retrived from the spreadsheet are **1625**. This means there are *at least* **199** non-corresponding tags between the two lists.

## Check duplicates in Zotero tags

Before performing any comparison between the two lists, we will check if there are any duplicates in the list downloaded from Zotero (we will take the `/tags` list as reference). Here is an example of how the entry for a tag is constructed.

In [13]:
tags[0]

{'tag': '_TODO',
 'links': {'self': {'href': 'https://api.zotero.org/groups/4822236/tags/_TODO',
   'type': 'application/json'},
  'alternate': {'href': 'http://zotero.org/groups/4822236/tags/_TODO',
   'type': 'text/html'}},
 'meta': {'type': 0, 'numItems': 3}}

In [14]:
seen = set()
duplicate_labels = set()

for tag in tags:
    label = tag["tag"]
    if label not in seen:
        seen.add(label)
    else:
        duplicate_labels.add(label)

print("Number of duplicates: ", str(len(duplicate_labels)))
print("==========================")
for duplicate in duplicate_labels:
    print(duplicate)

Number of duplicates:  84
big data
Technology
social
Electronic data processing
EDUCATION
digital
Arts
Methodology
Colleges and Universities
ideas
Classics
TEDTalks
NEH
Art / General
scholarship
Information access
History Teaching
office of digital humanities
Civilization
Data
Vol. 2, No. 1 Winter 2012
Humanities
sustainability
Media
TEACHING
Collaboration
Hypertext
Research
ODH
World history
Canada
topic modeling
Biology
History
annotation
Boekgeschiedenis
Distant Reading
Digital libraries
method
Technological innovations
humanidades digitales
Text Mining
Open Access
Philosophy
digital humanities
Social Science / Media Studies
Uncategorized
Manuscripts
⚠️ Invalid DOI
Electronic records
George Mason University
XML
cultural studies
Design
Internet
JISC
Electronic publishing
research
national endowment
Documents
Grafische industrie
text mining
American history
TEI
Letterkunde
Historians
educational
Statistics
Software
Music
Franse letterkunde
imaging
Education
Digital Humanities
Historio

In [15]:
duplicates = []

for tag in tags:
    tag_dict = {}
    if tag["tag"] in duplicate_labels:
        tag_dict = {"tag": tag["tag"],
                    "type": tag["meta"]["type"],
                    "numItems": tag["meta"]["numItems"],
                    "url_api": tag["links"]["self"]["href"],
                    "url_gui": tag["links"]["alternate"]["href"]}
        duplicates.append(tag_dict)

print(len(duplicates))

168


In [16]:
duplicates_df = pd.DataFrame.from_dict(duplicates)
duplicates_df

,tag,type,numItems,url_api,url_gui
0,/unread,0,6,https://api.zotero.org/groups/4822236/tags/%2F...,http://zotero.org/groups/4822236/tags/%2Funread
1,/unread,1,433,https://api.zotero.org/groups/4822236/tags/%2F...,http://zotero.org/groups/4822236/tags/%2Funread
2,⚠️ Invalid DOI,1,5,https://api.zotero.org/groups/4822236/tags/%E2...,http://zotero.org/groups/4822236/tags/%E2%9A%A...
3,⚠️ Invalid DOI,0,1,https://api.zotero.org/groups/4822236/tags/%E2...,http://zotero.org/groups/4822236/tags/%E2%9A%A...
4,American history,0,2,https://api.zotero.org/groups/4822236/tags/Ame...,http://zotero.org/groups/4822236/tags/American...
...,...,...,...,...,...
163,"Vol. 2, No. 1 Winter 2012",1,1,https://api.zotero.org/groups/4822236/tags/Vol...,http://zotero.org/groups/4822236/tags/Vol.+2%2...
164,World history,0,1,https://api.zotero.org/groups/4822236/tags/Wor...,http://zotero.org/groups/4822236/tags/World+hi...
165,World history,1,1,https://api.zotero.org/groups/4822236/tags/Wor...,http://zotero.org/groups/4822236/tags/World+hi...
166,XML,0,9,https://api.zotero.org/groups/4822236/tags/XML,http://zotero.org/groups/4822236/tags/XML


As one can see here, there are several "apparent" duplicates. The same tag can be found once with `type = 0`, once with `type = 1`. `type = 1` means that the tag was added automatically when the item was imported into the Zotero library. The counts for `type = 0` and `type = 1` are different, but when added together they sum up to the count found for each tag in the web GUI.

Let's create a new list of tags from Zotero without duplicates. We will then use the list for the purpose of comparison with the spreadsheet list.

In [17]:
print(len(tags_list))
tags_Zotero_dedup = list(set(tags_list))
print(len(tags_Zotero_dedup))
print(len(tags_list)-len(tags_Zotero_dedup), "duplicates removed")

1824
1740
84 duplicates removed


## Check duplicates in spreadsheet tags

In [18]:
from collections import Counter
counter = Counter(tags_spreadsheet)
duplicates_spreadsheet = {tag: count for tag, count in counter.items() if count > 1}
print("Tags that appear more than once:", len(duplicates_spreadsheet))
print("====================================")
for tag, count in duplicates_spreadsheet.items():
    print(tag, "[Count: " + str(count) + "]") 

Tags that appear more than once: 31
(teil-)automatisch generiert [Count: 2]
academics [Count: 2]
arts [Count: 2]
computer-mediated communication [Count: 2]
computers [Count: 2]
criticism/ data processing [Count: 2]
data/databases [Count: 2]
database [Count: 2]
digitalhumanities [Count: 2]
digitize [Count: 2]
digitized [Count: 3]
dokumentation [Count: 2]
earth sciences [Count: 2]
editionswissenschafte [Count: 2]
education, higher [Count: 2]
encoding [Count: 2]
estadísticas lexicales [Count: 2]
french literature [Count: 2]
handschriften. epigrafie. paleografie [Count: 3]
higher education [Count: 2]
humanidades digitales [Count: 2]
interdisciplinariedad [Count: 2]
interfaces (informatique) [Count: 2]
invisibleaustralians [Count: 2]
manuscrit [Count: 2]
minería de datos [Count: 2]
scholars [Count: 2]
the arts [Count: 2]
computers / general [Count: 2]
databases [Count: 2]
higher ed [Count: 2]


Let's now delete the duplicates from the list.

In [19]:
print(len(tags_spreadsheet))
tags_spreadsheet_dedup = list(set(tags_spreadsheet))
print(len(tags_spreadsheet_dedup))
print(len(tags_spreadsheet)-len(tags_spreadsheet_dedup), "duplicates removed")

1622
1589
33 duplicates removed


We now have two lists, `tags_Zotero_dedup` and `tags_spreadsheet_dedup`, which we can use for comparison.

## Compare Zotero and spreadsheet tags

Let's get a summary of the number of elements in both lists.

In [20]:
print(len(tags_Zotero_dedup))
print(len(tags_spreadsheet_dedup))

1740
1589


Checking tags that are present in the Zotero library but not present in the spreadsheet.

In [21]:
def compare_lists(list1, list2):
    differences = []
    for item in list1:
        if item not in list2:
            differences.append(item)
    print(len(differences))
    return differences

In [22]:
extra_tags_in_Zotero = compare_lists(tags_Zotero_dedup, tags_spreadsheet_dedup)

1180


In [23]:
extra_tags_in_spreadsheet = compare_lists(tags_spreadsheet_dedup, tags_Zotero_dedup)

1029


## Normalizing tags in Zotero

### Creating the mapping

We will now create a dictionary where each key corresponds to a tag in Zotero. For each tag, its normalized version is given as value of the key. The list of values should eventually correspond to the tags extracted from the spreadsheet.

This dictionary should serve as a mapping for applying the corrections.

In [24]:
mapping = {item: None for item in sorted(tags_Zotero_dedup)}

We will also reassign `tags_spreadsheet` to `tags_spreadsheet_dedup` to make things easier.

In [25]:
tags_spreadsheet = tags_spreadsheet_dedup

Let's also create a custom function to compare the two lists of tags directly.

In [26]:
def compare_Zotero_spreadsheet():
    extra_tags_in_Zotero = compare_lists(mapping.values(), tags_spreadsheet)
    extra_tags_in_spreadsheet = compare_lists(tags_spreadsheet, mapping.values())
    return extra_tags_in_Zotero, extra_tags_in_spreadsheet

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

1740
1589


### Lowercase

As one can see, there is a huge difference between the tags in Zotero. This is probably because tags in the spreadsheet were all lowercased.

So, let's try the comparison again by lowercasing the tags in `tags_list`.

In [27]:
for tag in mapping:
    mapping[tag] = tag.lower()

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

262
170


### Prefixes

As one can see from the following analysis, many tags in Zotero have prefixes that have been deleted in the spreadsheet.

In [28]:
tags_with_prefixes = []

for tag in extra_tags_in_Zotero:
    if '_' in tag or ':' in tag or '-' in tag:
        tags_with_prefixes.append(tag)

for tag in tags_with_prefixes:
    print(tag)

#broken_attachments
'2001: a space odyssey' (film)
activity: annotate
digital libraries -- access control
diskurs analyse - literatur -  informatik
f-cultural heritage
f-history
giraudoux, jean (1882-1944) -- language
giraudoux, jean (1882-1944) -- langue
history -- research
history -- sources
history -- study & teaching
humanities -- digital libraries
linguistique -- informatique
littérature française -- 19e siècle -- thèmes, motifs
littérature française -- commentaire de texte
meta: theorizing
object: code
object: data
object: digital humanities
object: infrastructures
object: language
object: metadata
object: people
object: texts
récits de voyages français -- 19e siècle -- histoire et critique
statistique lexicale -- informatique
united states -- social life & customs -- to 1775
uni: bielefeld
uni: münchen
uni: stuttgart
uni: tübingen (eberhard karls universität)
voyages -- dans la littérature
_todo
act_annotating
act_archiving
act_collaborating
act_communicating
act_conceptualizing

There are several prefixes that were deleted in the spreadsheet. We can make a list of them:

In [29]:
prefixes = []

pattern = r'^[A-Za-z]+(-|_|: )'

for tag in tags_with_prefixes:
    if result := re.match(pattern, tag):
        prefixes.append(str(result.group()))

prefixes = sorted(list(set(prefixes)))
print(len(prefixes))
print("========")
for item in prefixes:
    print(item)

16
act_
activity: 
f-
field: 
goal_
meta: 
meta_
ob_
obj_
object: 
special: 
supervisor: 
t_
type: 
uni: 
university: 


Now we can delete these prefixes and re-run the comparison.

In [30]:
for original in mapping.keys():
    for prefix in prefixes:
        mapping[original] = re.sub(f'^{prefix}', '', mapping[original])

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

37
29


### Delete quotes

In [31]:
tags_with_quotes = [tag for tag in extra_tags_in_Zotero if "'" in tag]
for tag in tags_with_quotes:
    print(tag)

'2001: a space odyssey' (film)
'american historical review'
'history matters'
'journal of american history'
'outline of american history'
'whole earth catalog'
'wikipedia'
artists' books
social science / women's studies
women's history


In [32]:
for original, normalized in mapping.items():
        mapping[original] = normalized.replace("'","")

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

27
20


### Delete semicolon

In [33]:
for original, normalized in mapping.items():
        mapping[original] = normalized.replace(";","")

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

21
18


### Remove "-- "

In [34]:
for original, normalized in mapping.items():
        mapping[original] = normalized.replace("-- ","")

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

7
4


### Remove other characters

In [35]:
pattern = r'^(/|_|⚠️ )'

for original in mapping.keys():
    mapping[original] = re.sub(r'^(/|_|⚠️ )', '', mapping[original])

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

4
1


In [36]:
for tag in extra_tags_in_Zotero:
    print(tag)

#broken_attachments
diskurs analyse - literatur -  informatik
foreign language study / german
frankfurter buchmesse 2016


### Remove double space after "-"

In [37]:
for original, normalized in mapping.items():
        mapping[original] = normalized.replace("-  ","- ")

extra_tags_in_Zotero, extra_tags_in_spreadsheet = compare_Zotero_spreadsheet()

3
0


In [38]:
for tag in extra_tags_in_Zotero:
    print(tag)

#broken_attachments
foreign language study / german
frankfurter buchmesse 2016


Regarding all the tags that still do not correspond to the spreadsheet:
* `#broken_attachments` seems to be a tag internal to Zotero that is not even visibile in the GUI
* `foreign language study / german` is not actually used in the library (see above) (the tag does not even appear when extracting tags from the full JSON dump describing all the bibliographic items)
* `frankfurter buchmesse 2016` refers to a publication added after the clean-up was done (4th March 2024)